In [7]:
import csv

import networkx as nx
import numpy as np

import feather as ft
import seaborn as sns; sns.set()
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter
from operator import itemgetter
from os import listdir
from os.path import isfile, isdir, join, exists
from sklearn.preprocessing import MinMaxScaler

In [8]:
mypath = '../../log/'
dirs = [join(mypath, f) for f in listdir(mypath) if isdir(join(mypath,f)) and "Powerlaw" in f ]
#dirs = [join(mypath, f) for f in listdir(mypath) if isdir(join(mypath,f)) and ("Graph" in f or "URV" in f)]
dirs

['../../log/TestingPowerlaw']

In [9]:
G = nx.Graph()
with open('../../src/datasets/EmailURV/EmailURV.csv', 'r') as file:
    reader = csv.reader(file, delimiter=",")
    for row in reader:
        G.add_edge(int(row[0]), int(row[1]))
N_AGENTS = len(G.nodes())

In [11]:
time = 200

In [28]:
for ddir in dirs:
    kdirs = [f for f in listdir(ddir) if isdir(join(ddir,f))]
    kcores = [join(join(ddir, f),'k_core.csv') for f in kdirs]
rawdataframe = pd.read_csv(kcores[0], sep=',', skiprows=12)

In [52]:
agent = int(rawdataframe.loc[
    rawdataframe['time']==1
].loc[rawdataframe['state']=='i']['agent'])
kcore = int(rawdataframe.loc[
    rawdataframe['time']==1
].loc[rawdataframe['state']=='i']['k-core'])

In [29]:
rawdataframe

,time,agent,k-core,state,degree
0,1,0,0,s,0
1,1,1,10,s,30
2,1,2,10,s,23
3,1,3,10,s,38
4,1,4,9,s,22
5,1,5,8,s,10
6,1,6,10,s,21
7,1,7,10,s,26
8,1,8,7,s,8
9,1,9,10,s,17


Add column of initial spreader

In [53]:
rawdataframe['k-core-init'] = kcore
rawdataframe['agent-init'] = agent

In [54]:
rawdataframe

,time,agent,k-core,state,degree,k-core-init,agent-init
0,1,0,0,s,0,5,1003
1,1,1,10,s,30,5,1003
2,1,2,10,s,23,5,1003
3,1,3,10,s,38,5,1003
4,1,4,9,s,22,5,1003
5,1,5,8,s,10,5,1003
6,1,6,10,s,21,5,1003
7,1,7,10,s,26,5,1003
8,1,8,7,s,8,5,1003
9,1,9,10,s,17,5,1003


In [34]:
count_state = rawdataframe.groupby(['time', 'state'])[['state']].apply(len)

In [36]:
count_state.reset_index()

,time,state,0
0,1,i,1
1,1,s,1133
2,2,i,1
3,2,s,1133
4,3,i,2
5,3,s,1132
6,4,i,3
7,4,s,1131
8,5,i,6
9,5,s,1128
